In [5]:
import numpy as np
import pandas as pd
import math
from astropy.io import fits
from IPython.display import clear_output

annulus = 11
apeture = 4

numStars = 7057
data, header = fits.getdata('C:/Users/ronald.sumners/Desktop/KeplerSept17/Registered/6785753_32924944_f_reg.fits', header = True)

starData = pd.read_csv('./KeplerSept17/K01Extract_Sept17.csv')
starData.columns = starData.columns.str.replace(' ', '')
starData = starData.set_index('#')

x = []
y = []
starName = starData['Name'][:numStars]
 
for i in range(numStars):
    name1 = starName.iloc[i]
    x.append(starData['X'][starData['Name'] == name1].reset_index(drop=True)[0])
    y.append(starData['Y'][starData['Name'] == name1].reset_index(drop=True)[0])
    clear_output(wait=True)
    print(f'PROGRESS: {(i/numStars)*100:.1f}%')


PROGRESS: 100.0%


In [6]:
totalcheck = 0
exclude = []
for i in range(numStars):
    centery = int(x[i])
    centerx = int(y[i])
    segement = np.array(data[centerx-annulus:centerx+annulus+1, centery-annulus:centery+annulus+1])

    mask = np.ones((2*annulus+1, 2*annulus+1), dtype = bool)
    mask[(int(annulus) - apeture):(int(annulus) + 1 + apeture), (int(annulus) - apeture):(int(annulus) + 1 + apeture)] = False

    annulus_data = segement[mask]

    check = ((annulus_data < 20000).sum() == annulus_data.size).astype(int)
    totalcheck += check
    
    if check == 0:
        name = starName.iloc[i]
        exclude.append(name)
print(totalcheck)

exclusions = pd.DataFrame(data = {'Names': exclude})
exclusions.to_csv('FilteringExclusions.csv', index=False)

4362


In [7]:
centerx = 3471
centery = 2140
segement = np.array(data[centerx-annulus:centerx+annulus+1, centery-annulus:centery+annulus+1])

mask = np.ones((2*annulus+1, 2*annulus+1), dtype = bool)
mask[(int(annulus) - apeture):(int(annulus) + 1 + apeture), (int(annulus) - apeture):(int(annulus) + 1 + apeture)] = False

annulus_data = segement[mask]

check = ((annulus_data < 20000).sum() == annulus_data.size).astype(int)
totalcheck += check
print(check)

1


In [8]:
exclusions = pd.read_csv('FilteringExclusions.csv')['Names'].to_numpy()
print(exclusions)
print(len(exclusions))

starData = pd.read_csv('./KeplerSept17/K01Extract_Sept17.csv')
print(starData.shape)
starData = starData[~starData['Name'].isin(exclusions)]
print(starData.shape)

starData.to_csv('K01ExtractFiltered.csv', index=False)
print('Complete.')

['K01-3' 'K01-5' 'K01-6' ... 'K01-9616' 'K01-9619' 'K01-9625']
2695
(246995, 43)
(152670, 43)
Complete.
